<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/annotation_import/image.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>




<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/annotation_import/image.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Annotation Import
This notebook will provide examples of each supported annotation type for image assets. 

### [Model-assisted labeling (MAL)](https://docs.labelbox.com/v4/docs/model-assisted-labeling)
* This workflow allows you to import computer-generated predictions (or simply annotations created outside of Labelbox) as pre-labels on an asset. 
The imported annotations will be pre-populated in the labeling editor. However, in order to convert the pre-labels to real annotations, a human labeler will still need to open the Data Row in the Editor and submit it. This functionality is designed to speed up human labeling.


### [Import ground truth](https://docs.labelbox.com/v4/docs/import-ground-truth)
* This  workflow functionality allows you to bulk import your ground truth annotations from an external or third-party labeling system into Labelbox Annotate. Using the label import API to import external data is a useful way to consolidate and migrate all annotations into Labelbox as a single source of truth.


### Python Annotation types vs NDJSON
**Python Annotation Type (recommended)**
- Provides a seamless transition between third-party platforms, machine learning pipelines, and Labelbox.

- Allows you to build annotations locally with local file paths, numpy arrays, or URLs

- Easily convert Python Annotation Type format to NDJSON format to quickly import annotations to Labelbox

- It supports one-level nested classification (free text / radio / checklist) under the object or classification annotation.

**NDJSON**
- Skip formatting annotation payload in the Python Annotation Types format just to convert back to NDJSON

- Ability to create the payload in the NDJSON import format directly

- It supports any levels of nested classification (free text / radio / checklist) under the object or classification annotation.

## Imports

In [ ]:
!pip install -q 'labelbox[data]'

In [ ]:
import labelbox as lb
import labelbox.types as lb_types
import uuid
import numpy as np


## Replace with your API KEY

Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
API_KEY = ""
client = lb.Client(API_KEY)

## Supported Annotations for Image


### Supported Annotation Types

In [ ]:
########### Radio Classification ###########

# Python annotation
radio_annotation = lb_types.ClassificationAnnotation(
    name="radio_question", 
    value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "second_radio_answer"))
)


# NDJSON
radio_annotation_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer'}
} 

In [ ]:
########## Nested Radio and Checklist Classification is only supported with NDJSON tools##########

# NDJSON 
nested_radio_annotation_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer"},
  "classifications" : [
   {'name': 'sub_radio_question', 'answer': {'name': 'first_sub_radio_answer'}}
   ]
}


nested_checklist_annotation_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer", 
      "classifications" : [
        {
          "name": "sub_checklist_question", 
          "answer": {"name": "first_sub_checklist_answer"}
        }          
      ]         
  }]
}


In [ ]:
############ Checklist question ############

# Python Annotations
checklist_annotation= lb_types.ClassificationAnnotation(
  name="checklist_question", # must match your ontology feature's name
  value=lb_types.Checklist(
      answer = [
        lb_types.ClassificationAnswer(
            name = "first_checklist_answer"
        ), 
        lb_types.ClassificationAnswer(
            name = "second_checklist_answer"
        )
      ]
    )
 )

# NDJSON
checklist_annotation_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer'},
    {'name': 'second_checklist_answer'}
  ]
}

In [ ]:
############# Free text Classification #############

# Python annotation
text_annotation = lb_types.ClassificationAnnotation(
  name="free_text",  # must match your ontology feature's name
  value=lb_types.Text(answer="sample text")
)


# NDJSON
text_annotation_ndjson = {
  'name': 'free_text',
  'answer': 'sample text',
}

In [ ]:
####### Bounding box #######


# Python Annotation 
bbox_annotation = lb_types.ObjectAnnotation(
  name = "bounding_box",  # must match your ontology feature's name
  value = lb_types.Rectangle(
        start=lb_types.Point(x=977, y=1690), # Top left
        end=lb_types.Point(x=330, y=225), # Bottom right
    )
)

#NDJSON 
bbox_annotation_ndjson = {
  'name': 'bounding_box',
  'bbox': {
          "top": 977,
          "left": 1690,
          "height": 330,
          "width": 225
      }
}


In [ ]:
# Bounding box with nested classification
bbox_with_radio_subclass_annotation = lb_types.ObjectAnnotation(
    name="bbox_with_radio_subclass",
    value=lb_types.Rectangle(
        start=lb_types.Point(x=933, y=541), # Top left
        end=lb_types.Point(x=191, y=330), # Bottom right
    ),
    classifications=[
    	lb_types.ClassificationAnnotation(
        	name="sub_radio_question",
      		value=lb_types.Radio(answer=lb_types.ClassificationAnswer(name="first_sub_radio_answer"))
    )
  ]
)


## NDJSON
bbox_with_radio_subclass_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "classifications": [{
        "name": "sub_radio_question",
        "answer": 
            { "name":"first_sub_radio_answer" }
         
    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}

In [ ]:
########## Polygon ##########
# Python AnotationTypes 
polygon_annotation = lb_types.ObjectAnnotation(
  name = "polygon",  # must match your ontology feature's name 
  value=lb_types.Polygon( # Coordinates for the verticies of your polygon
        points=[lb_types.Point(x=1489.581,y=183.934), lb_types.Point(x=2278.306,y=256.885), lb_types.Point(x=2428.197,y=200.437), lb_types.Point(x=2560.0,y=335.419),
                lb_types.Point(x=2557.386,y=503.165), lb_types.Point(x=2320.596,y=503.103), lb_types.Point(x=2156.083, y=628.943), lb_types.Point(x=2161.111,y=785.519),
                lb_types.Point(x=2002.115, y=894.647), lb_types.Point(x=1838.456,y=877.874), lb_types.Point(x=1436.53,y=874.636), lb_types.Point(x=1411.403,y=758.579),
                lb_types.Point(x=1353.853,y=751.74), lb_types.Point(x=1345.264, y=453.461), lb_types.Point(x=1426.011,y=421.129)]
    ),
)



# NDJSON

polygon_annotation_ndjson = {
  'name': 'polygon',
  'polygon': [
    {'x': 1489.581, 'y': 183.934},
    {'x': 2278.306, 'y': 256.885},
    {'x': 2428.197, 'y': 200.437},
    {'x': 2560.0, 'y': 335.419},
    {'x': 2557.386, 'y': 503.165},
    {'x': 2320.596, 'y': 503.103},
    {'x': 2156.083, 'y': 628.943},
    {'x': 2161.111, 'y': 785.519},
    {'x': 2002.115, 'y': 894.647},
    {'x': 1838.456, 'y': 877.874},
    {'x': 1436.53, 'y': 874.636},
    {'x': 1411.403, 'y': 758.579},
    {'x': 1353.853, 'y': 751.74},
    {'x': 1345.264, 'y': 453.461},
    {'x': 1426.011, 'y': 421.129},
    {'x': 1489.581, 'y': 183.934}
  ]
}

In [ ]:
######### Mask #########


# Python 
# Identifying what values in the numpy array correspond to the mask annotation
color = (0, 0, 0)

# convert a polygon to mask
im_height, im_width = 100,100 #need to provide the height and width of image.
mask_data = lb_types.MaskData(arr=
                     polygon_annotation.value.draw(height=im_height,width=im_width,color=color))

# convert a 2D array to 3D array
arr_2d = np.zeros((100,100), dtype='uint8')
mask_data = lb_types.MaskData.from_2D_arr(arr_2d)

# a 3D array where 3rd axis is RGB values.
mask_data = lb_types.MaskData(arr= np.zeros([400,450,3],dtype='uint8'))

mask_annotation = lb_types.ObjectAnnotation(
  name = "mask", # must match your ontology feature's name
  value=lb_types.Mask(mask=mask_data, color=color),
)


# NDJSON
mask_annotation_ndjson = {
  'name': 'mask',
  'classifications': [],
  'mask': {'instanceURI': 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA',
  'colorRGB': (0, 0, 0)}
}



In [ ]:
######## Point Annotation ########

# Python Annotation
point_annotation = lb_types.ObjectAnnotation(
  name = "point",  # must match your ontology feature's name
  value = lb_types.Point(x=1166.606, y=1441.768),
)

# NDJSON
point_annotation_ndjson = {
  'name': 'point',
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

In [ ]:
###### Polygon ######


# Python Annotation 

polyline_annotation = lb_types.ObjectAnnotation(
  name = "polyline", # must match your ontology feature's name
  value=lb_types.Line( # Coordinates for the keypoints in your polyline
        points=[lb_types.Point(x=2534.353, y=249.471), lb_types.Point(x=2429.492, y=182.092), lb_types.Point(x=2294.322, y=221.962), lb_types.Point(x=2224.491, y=180.463), lb_types.Point(x=2136.123, y=204.716),
                lb_types.Point(x=1712.247, y=173.949), lb_types.Point(x=1703.838, y=84.438), lb_types.Point(x=1579.772, y=82.61), lb_types.Point(x=1583.442, y=167.552),
                lb_types.Point(x=1478.869, y=164.903), lb_types.Point(x=1418.941, y=318.149), lb_types.Point(x=1243.128, y=400.815), lb_types.Point(x=1022.067, y=319.007),
                lb_types.Point(x=892.367, y=379.216), lb_types.Point(x=670.273, y=364.408), lb_types.Point(x=613.114, y=288.16), lb_types.Point(x=377.559, y=238.251),
                lb_types.Point(x=368.087, y=185.064), lb_types.Point(x=246.557, y=167.286), lb_types.Point(x=236.648, y=285.61), lb_types.Point(x=90.929, y=326.412)]
    ),
)

# NDJSON
polyline_annotation_ndjson = {
  'name': 'polyline',
  'classifications': [],
  'line': [
    {'x': 2534.353, 'y': 249.471},
    {'x': 2429.492, 'y': 182.092},
    {'x': 2294.322, 'y': 221.962},
    {'x': 2224.491, 'y': 180.463},
    {'x': 2136.123, 'y': 204.716},
    {'x': 1712.247, 'y': 173.949},
    {'x': 1703.838, 'y': 84.438},
    {'x': 1579.772, 'y': 82.61},
    {'x': 1583.442, 'y': 167.552},
    {'x': 1478.869, 'y': 164.903},
    {'x': 1418.941, 'y': 318.149},
    {'x': 1243.128, 'y': 400.815},
    {'x': 1022.067, 'y': 319.007},
    {'x': 892.367, 'y': 379.216},
    {'x': 670.273, 'y': 364.408},
    {'x': 613.114, 'y': 288.16},
    {'x': 377.559, 'y': 238.251},
    {'x': 368.087, 'y': 185.064},
    {'x': 246.557, 'y': 167.286},
    {'x': 236.648, 'y': 285.61},
    {'x': 90.929, 'y': 326.412}
  ]
}


In [ ]:
##### Relationship ##### 
# only supported with NDJson and for MAL imports 

uuid_source = str(uuid.uuid4())
uuid_target = str(uuid.uuid4())

bbox_source = {
  'name': 'bounding_box',
  'uuid': uuid_source,
  'bbox': {
          "top": 177,
          "left": 690,
          "height": 130,
          "width": 125
      }
}

bbox_target = {
  'name': 'bounding_box',
  'uuid': uuid_target,
  'bbox': {
          "top": 277,
          "left": 590,
          "height": 230,
          "width": 325
      }
}

relationship_annotation_ndjson = {
    "name": "relationship", 
    "relationship": {
      "source": uuid_source,
      "target": uuid_target,
      "type": "unidirectional"
    }
}


# Upload Annotations - putting it all together


## Step 1: Import data rows into Catalog



In [ ]:
# send a sample image as batch to the project
global_key="2560px-Kitano_Street_Kobe01s5s41102.jpeg"

test_img_url = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg" ,
    "global_key": global_key
}
  

dataset = client.create_dataset(name="demo_dataset_img")
task = dataset.create_data_rows([test_img_url])
task.wait_till_done()
print("Errors:",task.errors)
print("Failed data rows:", task.failed_data_rows)

There are errors present. Please look at `task.errors` for more details


Errors: Duplicate global keys found: 2560px-Kitano_Street_Kobe01s5s41102.jpeg
Failed data rows: [{'message': 'Duplicate global keys found: 2560px-Kitano_Street_Kobe01s5s41102.jpeg', 'failedDataRows': [{'globalKey': '2560px-Kitano_Street_Kobe01s5s41102.jpeg', 'rowData': 'https://storage.googleapis.com/labelbox-datasets/image_sample_data/2560px-Kitano_Street_Kobe01s5s4110.jpeg', 'attachmentInputs': []}]}]


## Step 2: Create/select an Ontology
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the `name` fields in your annotations to ensure the correct feature schemas are matched.

For example, when we create the bounding box annotation above, we provided the `name` as `bounding_box`. Now, when we setup our ontology, we must ensure that the name of my bounding box tool is also `bounding_box`. The same alignment must hold true for the other tools and classifications we create in our ontology.




In [ ]:
ontology_builder = lb.OntologyBuilder(
  classifications=[ # List of Classification objects
    lb.Classification( 
      class_type=lb.Classification.Type.RADIO,
      name="radio_question", 
      options=[
        lb.Option(value="first_radio_answer"),
        lb.Option(value="second_radio_answer")
      ]
    ),
    lb.Classification(
      class_type=lb.Classification.Type.CHECKLIST,
      name="checklist_question", 
      options=[
        lb.Option(value="first_checklist_answer"),
        lb.Option(value="second_checklist_answer")
      ]
    ), 
    lb.Classification( 
      class_type=lb.Classification.Type.TEXT,
      name="free_text"
    ),
    lb.Classification(
        class_type=lb.Classification.Type.RADIO,
        name="nested_radio_question",
        options=[
            lb.Option("first_radio_answer",
                options=[
                    lb.Classification(
                        class_type=lb.Classification.Type.RADIO,
                        name="sub_radio_question",
                        options=[lb.Option("first_sub_radio_answer")]
                    )
                ]
            )
          ] 
        ),
    lb.Classification(
      class_type=lb.Classification.Type.CHECKLIST,
      name="nested_checklist_question",
      options=[
          lb.Option("first_checklist_answer",
            options=[
              lb.Classification(
                  class_type=lb.Classification.Type.CHECKLIST,
                  name="sub_checklist_question", 
                  options=[lb.Option("first_sub_checklist_answer")]
              )
          ]
        )
      ]
    ),      
  ],
  tools=[ # List of Tool objects
    lb.Tool( 
      tool=lb.Tool.Type.BBOX,
      name="bounding_box"), 
    lb.Tool( 
      tool=lb.Tool.Type.BBOX,
      name="bbox_with_radio_subclass",
      classifications=[
            lb.Classification(
                class_type=lb.Classification.Type.RADIO,
                name="sub_radio_question",
                options=[
                  lb.Option(value="first_sub_radio_answer")
                ]
              ),
        ]
      ), 
    lb.Tool( 
      tool=lb.Tool.Type.POLYGON,
      name="polygon"),
    lb.Tool( 
      tool=lb.Tool.Type.SEGMENTATION,
      name="mask"),
 	  lb.Tool( 
      tool=lb.Tool.Type.POINT,
      name="point"), 
    lb.Tool( 
      tool=lb.Tool.Type.LINE,
      name="polyline"),
    lb.Tool( 
      tool=lb.Tool.Type.RELATIONSHIP,
      name="relationship")]
)

ontology = client.create_ontology("Image Prediction Import Demo", ontology_builder.asdict(), media_type=lb.MediaType.Image)

## Step 3: Create a labeling project

Connect the ontology to the labeling project

In [ ]:

# create a project and configure the ontology 
project = client.create_project(
    name="annotations_import_project_demo",
    media_type=lb.MediaType.Image,
    queue_mode=lb.QueueMode.Batch)

project.setup_editor(ontology) # Connect your ontology and editor to your MAL project

## Step 4: Send a batch of data rows to the project


In [ ]:
batch = project.create_batch(
    "Initial batch2", # name of the batch
    global_keys=global_key, # DataRow objects, a list of global keys or data row ids are supported
    priority=1 # priority between 1-5
)
print("Batch", batch)

Batch <Batch {
    "consensus_settings_json": "{\"numberOfLabels\":1,\"coveragePercentage\":0}",
    "created_at": "2023-03-17 13:58:44+00:00",
    "name": "Initial batch2",
    "size": 0,
    "uid": "d4b46230-c4cb-11ed-8ef7-03b2e7ba4cb0",
    "updated_at": "2023-03-17 13:58:44+00:00"
}>


## Step 5: Create the annotations payload

Create the annotations payload using the snippets of code above

Labelbox support two formats for the annotations payload: NDJSON and Python Annotation types. Both are described below. If you are using Python Annotation types, compose your annotations into Labels attached to the data rows.

### Python Annotations

Here we create the complete label ndjson payload of annotations only using python annotation format. There is one annotation for each reference to an annotation that we created.

In [ ]:
# create a Label

label = []

label.append(
    lb_types.Label(
        data=lb_types.ImageData(
            global_key=global_key),
        annotations = [
            checklist_annotation, 
            text_annotation,
            bbox_annotation, 
            bbox_with_radio_subclass_annotation, 
            polygon_annotation, 
            mask_annotation, 
            point_annotation, 
            polyline_annotation
        ]
    )
)


### NDJSON annotations
Here we create the complete label ndjson payload of annotations only using NDJSON format. There is one annotation for each reference to an annotation that we created above.

In [ ]:
label_ndjson_method2 = []
for annotation in [radio_annotation_ndjson, 
              checklist_annotation_ndjson, 
              text_annotation_ndjson,
              bbox_annotation_ndjson, 
              bbox_with_radio_subclass_ndjson, 
              polygon_annotation_ndjson, 
              mask_annotation_ndjson, 
              point_annotation_ndjson, 
              polyline_annotation_ndjson,
              nested_radio_annotation_ndjson,
              nested_checklist_annotation_ndjson,
              relationship_annotation_ndjson, ## Only supported for MAL imports 
              bbox_source,
              bbox_target
              ]:
  annotation.update({## Passing the uuid optional id to construct the relationship annotation
      'dataRow': {'globalKey':global_key},
  })
  label_ndjson_method2.append(annotation)


## Step 6: Upload annotations to a project as pre-labels or complete labels

### Model-Assisted Labeling
For the purpose of this tutorial only run one of the label_ndjosn  annotation type tools at the time (NDJSON or Annotation types). Delete the previous labels before uploading labels that use the 2nd method (ndjson)


In [ ]:
# Upload MAL label for this data row in project
upload_job = lb.MALPredictionImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="mal_job"+str(uuid.uuid4()), 
    predictions=label_ndjson_method2)

print("Errors:", upload_job.errors)
print("Status of uploads: ", upload_job.statuses)
print(" ")

Errors: []
 


### Label Import


In [ ]:
# Upload label for this data row in project
upload_job = lb.LabelImport.create_from_objects(
    client = client, 
    project_id = project.uid, 
    name="label_import_job"+str(uuid.uuid4()),  
    labels=label_ndjson_method2)

print("Errors:", upload_job.errors)
print("Status of uploads: ", upload_job.statuses)
print(" ")

In [ ]:
# project.delete()
# dataset.delete()